# Проект

#### Григорьев Максим

### Задание и описание данных 

С 1 октября НДС на бриллианты отменяется, что делает их новым инвестиционным инструментом. Давайте создадим для них модель ценообразования. Dataset содержит характеристики бриллиантов и цены на них с сайта jamesallen (B2C-платформа) по состоянию на 2022-07-01

**Описание характеристик**

* fluor - флюоресценция (свойство драгоценного камня светиться) 
* symmetry - показатель симметрии
* platform - название платформы, на которой был размещен драгоценный камень
* shape - форма
* color - цвет
* clarity - чистота
* cut - качество огранки (может быть только для круглых камней)
* polish - полировка
* id - номер драгоценного камня
* date - дата
* price - цена
* carat - количество каратов
* price_per_carat - цена за карат
* z - длина (диаметр)
* x - ширина
* depth_perc - отношение высоты к ширине
* y - высота

#### Подключение необходимых библиотек

In [23]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from category_encoders import TargetEncoder

from typing import Tuple, List

In [2]:
df = pd.read_csv('diamonds.csv')
df.head()

,Unnamed: 0,fluor,symmetry,platform,shape,color,clarity,cut,polish,id,date,price,carat,price_per_carat,z,x,depth_perc,y
0,135269,NONE,EX,jamesallen,PS,F,SI1,NaN,EX,13870838,202206,12850.0,1.55,8290.32,10.29,6.41,62.0,3.97420
1,48477,MED,EX,jamesallen,RD,H,VVS2,EX,EX,11725253,202207,7510.0,1.02,7362.75,6.41,6.45,62.5,4.03125
2,236786,NONE,EX,jamesallen,EM,H,IF,NaN,EX,14444347,202205,21220.0,2.01,10557.21,8.60,6.37,65.0,4.14050
3,235781,NONE,EX,jamesallen,RD,E,VS2,EX,EX,14438434,202207,8660.0,1.00,8660.00,6.39,6.44,61.4,3.95416
4,277744,NONE,VG,jamesallen,RD,F,VS1,VG,EX,14615276,202206,8480.0,1.00,8480.00,6.28,6.36,62.3,3.96228


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 222222 entries, 0 to 222221
Data columns (total 18 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   Unnamed: 0       222222 non-null  int64  
 1   fluor            222207 non-null  object 
 2   symmetry         222218 non-null  object 
 3   platform         222218 non-null  object 
 4   shape            222218 non-null  object 
 5   color            222218 non-null  object 
 6   clarity          222218 non-null  object 
 7   cut              148981 non-null  object 
 8   polish           222218 non-null  object 
 9   id               222222 non-null  int64  
 10  date             222222 non-null  int64  
 11  price            222222 non-null  float64
 12  carat            222222 non-null  float64
 13  price_per_carat  222222 non-null  float64
 14  z                222222 non-null  float64
 15  x                222222 non-null  float64
 16  depth_perc       222222 non-null  floa

### Регрессия

#### Задание 1: Очистка данных
Не все драгоценные камни удается продать в течение месяца, поэтому в таблице есть повторы. 

Объединим данные по каждому самоцвету и найдем аномалии в данных. Также убедимся, что другие параметры драгоценного камня не меняются.

In [4]:
# Разделим выборку драгоценных камней на две части, встречающиеся 1 раз и которые встречались 2 и более раза
df_one = df.query('id.duplicated(keep=False) == False')
df_three = df.query('id.duplicated(keep=False) == True')

# Удалим аномальные значения и выбросы
df_three = df_three.query('price.diff().abs() <= 0.5 * price.mean()', engine='python').sort_values(by=['id', 'date'])

# Возьмем в качестве цены последнее по времени значение
df_three = df_three.groupby('id').agg('last').reset_index()

# Объединим, чтобы получить результат
df = pd.concat([df_one, df_three]).reset_index(drop=True)

Цены на бриллианты колеблются довольно значительно, попробуем рассчитать уровень инфляции и скорректировать цены на последний месяц. Следует отметить, что не все цены на бриллианты изменяются синхронно, то есть изменение в отдельных группах может быть разным.

In [5]:
# Определим группы, по которым будет сегментироваться инфляция
categories = ['color', 'clarity', 'carat']  # примеры категорий, включая группу каратов

# Ценовой индекс
df_index = df.groupby(['date'] + categories)[['price']].mean().reset_index()

# Сопоставление групп с максимальной датой
date_max = df_index.date.max()  # последняя дата
df_index = df_index.merge(
    df_index.query('date == @date_max')[categories + ['price']].rename(columns={'price': 'price_per_carat_max'}),
    on=categories, how='outer'
)

# Расчет инфляции
df_index['inflation'] = df_index['price_per_carat_max'] / df_index['price']

# Объединение всех данных в одну таблицу
df_with_inf = df.merge(df_index[['date'] + categories + ['inflation']], on=['date'] + categories, how='left')

In [6]:
df_with_inf.head()

,Unnamed: 0,fluor,symmetry,platform,shape,color,clarity,cut,polish,id,date,price,carat,price_per_carat,z,x,depth_perc,y,inflation
0,189202,NONE,EX,jamesallen,PS,D,SI2,NaN,EX,14224443,202205,9120.0,1.51,6039.74,10.67,6.23,59.0,3.67570,0.962390
1,76462,NONE,VG,jamesallen,EM,K,SI2,NaN,VG,12902293,202205,1390.0,0.91,1527.47,6.30,4.58,71.0,3.25180,0.881565
2,72397,NONE,VG,jamesallen,PS,J,VS1,NaN,VG,12771762,202207,15990.0,2.27,7044.05,11.66,7.26,62.2,4.51572,1.000000
3,269475,MED,EX,jamesallen,RD,J,SI2,EX,EX,14584509,202205,3700.0,1.01,3663.37,6.37,6.42,62.0,3.98040,1.079663
4,321928,NONE,EX,jamesallen,RD,G,SI1,EX,EX,14773748,202206,7170.0,1.00,7170.00,6.30,6.35,63.7,4.04495,1.054515


#### Задание 2: Модель

Определим функцию потерь (MSE или MAE). Построим baseline и линейную модель.

Я использую Mean Absolute Error (MAE), так как она измеряет среднее абсолютное отклонение предсказаний от реальных значений. MAE показывает, насколько в среднем ошибается модель в предсказаниях, независимо от направления ошибки (переподсчёт или недоподсчёт). Это делает MAE интуитивно понятной метрикой, особенно когда важна точность предсказаний без сильного влияния на большие выбросы.

Обоснование:

1. MAE легко интерпретировать, поскольку она выражается в тех же единицах, что и целевая переменная.
2. MAE устойчива к выбросам, так как ошибки взвешиваются равномерно, без усиления их влияния.
3. Выбросы не должны существенно влиять на результат.

Таким образом, MAE — оптимальный выбор для нашей задачи, где разброс цен на бриллианты значителен, но нас интересует точное среднее отклонение.

In [7]:
df_save = df.copy()

In [8]:
# Кодирование категориальных признаков
label_encoders = {}
categorical_features = ['fluor', 'symmetry', 'platform', 'shape', 'color', 'clarity', 'cut', 'polish']
for col in categorical_features:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    label_encoders[col] = le

In [9]:
# Разделение данных
X = df.drop(columns=["price"])  # исключаем целевую переменную
y = df["price"]  # целевая переменная
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=13)

In [10]:
# Baseline: предсказание среднего значения
baseline_pred = [y_train.mean()] * len(y_valid)

# Оценка качества Baseline
baseline_mae = mean_absolute_error(y_valid, baseline_pred)
print(f"Baseline MAE: {baseline_mae}")

# Построение базовой линейной модели
model = LinearRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_valid)

# Оценка модели (MAE)
mae = mean_absolute_error(y_valid, y_pred)
print(f"Mean Absolute Error (MAE): {mae}")

Baseline MAE: 6663.962655329724
Mean Absolute Error (MAE): 2968.920111850262


In [11]:
df = df_save.copy()

Теперь попробуем OHE (One Hot Encoding) или TargetEncoder. Нормализуем данные. Поработаем отсутствующие значения.

In [12]:
# Обработка отсутствующих значений
def preprocess_missing_values(df: pd.DataFrame) -> pd.DataFrame:
    df['fluor'] = df['fluor'].fillna('NONE')
    df['platform'] = df['platform'].fillna('other')
    for col in categorical_features:
        df[col] = df[col].fillna(df[col].mode()[0])
        df[col] = df[col].astype('category')
    df = df.fillna(df.median(numeric_only=True))  # Заполняем численные пропуски медианой
    return df

df = preprocess_missing_values(df)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150661 entries, 0 to 150660
Data columns (total 18 columns):
 #   Column           Non-Null Count   Dtype   
---  ------           --------------   -----   
 0   Unnamed: 0       150661 non-null  int64   
 1   fluor            150661 non-null  category
 2   symmetry         150661 non-null  category
 3   platform         150661 non-null  category
 4   shape            150661 non-null  category
 5   color            150661 non-null  category
 6   clarity          150661 non-null  category
 7   cut              150661 non-null  category
 8   polish           150661 non-null  category
 9   id               150661 non-null  int64   
 10  date             150661 non-null  int64   
 11  price            150661 non-null  float64 
 12  carat            150661 non-null  float64 
 13  price_per_carat  150661 non-null  float64 
 14  z                150661 non-null  float64 
 15  x                150661 non-null  float64 
 16  depth_perc       150

##### TargetEncoder

In [14]:
# Функция для TargetEncoder
def apply_target_encoder(train: pd.DataFrame, valid: pd.DataFrame, columns: List[str], target: pd.Series) -> Tuple[pd.DataFrame, pd.DataFrame]:
    encoder = TargetEncoder(cols=columns)
    train_encoded = encoder.fit_transform(train, target)
    valid_encoded = encoder.transform(valid)
    return train_encoded, valid_encoded

# Целевая переменная и признаки
y = df['price']
X = df.drop(columns=['price'])

# Разделение данных на обучающую и тестовую выборки
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=13)

# Применение TargetEncoder
X_train_te, X_valid_te = apply_target_encoder(X_train, X_valid, categorical_features, y_train)

# Нормализация данных
scaler = StandardScaler()
X_train_te_scaled = pd.DataFrame(scaler.fit_transform(X_train_te), columns=X_train_te.columns)
X_valid_te_scaled = pd.DataFrame(scaler.transform(X_valid_te), columns=X_valid_te.columns)

# Линейная модель для TargetEncoder
model_te = LinearRegression()
model_te.fit(X_train_te_scaled, y_train)
y_pred_te = model_te.predict(X_valid_te_scaled)
mae_te = mean_absolute_error(y_valid, y_pred_te)

print(f"MAE (TargetEncoder): {mae_te}")

MAE (TargetEncoder): 3032.321746344459


##### OHE (One Hot Encoding)

In [15]:
# Функция для OHE
def OHE(df: pd.DataFrame, columns: List[str]) -> Tuple[pd.DataFrame, List[str]]:
    index = df.index
    one = OneHotEncoder(sparse_output=False, categories='auto')
    ohe = one.fit_transform(df[columns])
    col_names = one.get_feature_names_out(input_features=columns)
    df = df.drop(columns, axis=1)
    df = df.reset_index(drop=True)
    df = pd.concat([df, pd.DataFrame(ohe, columns=col_names)], axis=1)
    df = df.set_index(index)
    return df, list(col_names)


df_ohe, ohe_columns = OHE(df, categorical_features)

# Разделение данных
X = df_ohe.drop(columns=["price"])  # Целевой признак исключаем
y = df_ohe["price"]

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=13)

# Нормализация численных данных (после разделения)
scaler = StandardScaler()
X_train_ohe_scaled = pd.DataFrame(scaler.fit_transform(X_train), columns=X_train.columns)
X_valid_ohe_scaled = pd.DataFrame(scaler.transform(X_valid), columns=X_valid.columns)

# Построение линейной модели
model = LinearRegression()
model.fit(X_train_ohe_scaled, y_train)
y_pred = model.predict(X_valid_ohe_scaled)
mae_ohe = mean_absolute_error(y_valid, y_pred)

# Вывод результатов
print(f"MAE (OHE): {mae_ohe}")

MAE (OHE): 2167.210564628672


#### Выполним подготовку данных 

In [16]:
df, _ = OHE(df, categorical_features)

X = df.drop(columns=["price"])  # Целевой признак исключаем
y = df["price"]

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=13)

# Нормализация численных данных (после разделения)
scaler = StandardScaler()
X_train = pd.DataFrame(scaler.fit_transform(X_train), columns=X_train.columns)
X_valid = pd.DataFrame(scaler.transform(X_valid), columns=X_valid.columns)


### KNN

In [22]:
# Построение модели KNN
model = KNeighborsRegressor(n_neighbors=5)  # Количество соседей можно варьировать
model.fit(X_train, y_train)
y_pred = model.predict(X_valid)

# Оценка качества модели
mae = mean_absolute_error(y_valid, y_pred)
print(f"Mean Absolute Error (MAE) using KNN: {mae}")

Mean Absolute Error (MAE) using KNN: 1559.390369362493


### Деревья решений

In [18]:
# Построение модели Decision Tree
model = DecisionTreeRegressor(random_state=13)
model.fit(X_train, y_train)
y_pred = model.predict(X_valid)

# Оценка качества модели
mae = mean_absolute_error(y_valid, y_pred)
print(f"Mean Absolute Error (MAE) with Decision Tree: {mae}")

Mean Absolute Error (MAE) with Decision Tree: 104.27172866956492


### Ансамблевые методы

In [ ]:
# your code here

## Заключение

Сравните все модели, выберите лучшую и сделайте вывод о задании в целом.

<Ваш вывод здесь >